In [1]:
#setwd(file.path("/volume/carehf/multiome/out/"))

In [22]:
library(Seurat)
library(dplyr)
library(ggplot2)
library(Signac)
library(GenomeInfoDb)
library(BSgenome.Hsapiens.UCSC.hg38)
library(EnsDb.Hsapiens.v86) # for a set of annotations for hg38
library(viridis)

# Load in data and quality control filter

In [4]:
inputFiles_dir <- c(
    'D9_RV' = "/home/ubuntu/carehf/data_multiome/D9_RV/",
    'D2_RV' = "/home/ubuntu/carehf/data_multiome/D2_RV/",
    'D3_RV' = "/home/ubuntu/carehf/data_multiome/D3_RV/",
    'D1_RV' = "/home/ubuntu/carehf/data_multiome/D1_RV/",
    'HF6_RV' = "/home/ubuntu/carehf/data_multiome/HF6_RV/",
    'HF9_RV' = "/home/ubuntu/carehf/data_multiome/HF9_RV/",
    'HF1_RV' = "/home/ubuntu/carehf/data_multiome/HF1_RV/",
    'HF3_RV' = "/home/ubuntu/carehf/data_multiome/HF3_RV/"
    )
inputFiles_dir

sampleNames = names(inputFiles_dir)
sampleNames

D9_RV 
 "/home/ubuntu/carehf/data_multiome/D9_RV/" 
                                      D2_RV 
 "/home/ubuntu/carehf/data_multiome/D2_RV/" 
                                      D3_RV 
 "/home/ubuntu/carehf/data_multiome/D3_RV/" 
                                      D1_RV 
 "/home/ubuntu/carehf/data_multiome/D1_RV/" 
                                     HF6_RV 
"/home/ubuntu/carehf/data_multiome/HF6_RV/" 
                                     HF9_RV 
"/home/ubuntu/carehf/data_multiome/HF9_RV/" 
                                     HF1_RV 
"/home/ubuntu/carehf/data_multiome/HF1_RV/" 
                                     HF3_RV 
"/home/ubuntu/carehf/data_multiome/HF3_RV/"

[1] "D9_RV"  "D2_RV"  "D3_RV"  "D1_RV"  "HF6_RV" "HF9_RV" "HF1_RV" "HF3_RV"

In [5]:
out_dir <- '/volume/carehf/multiome/out/QC_preprocessing/'

In [6]:
filtered_list <- list()

In [9]:
for (i in seq_along(sampleNames)){
    sampleFile_rna <- paste0(inputFiles_dir[[i]],'filtered_feature_bc_matrix.h5')
    sampleFile_atac <- paste0(inputFiles_dir[[i]],'atac_fragments.tsv.gz')
    sampleName <- sampleNames[i]
    print(paste0('QC: ', sampleName))
    
    # extract RNA and ATAC data
    data <- Read10X_h5(sampleFile_rna)
    rna_counts <- data$'Gene Expression'
    atac_counts <- data$Peaks
    
    # Create seurat object from RNA gene expression
    seurat <- CreateSeuratObject(counts = rna_counts, project = sampleName)
    
    # Add in the ATAC-seq data. We'll only use peaks in standard chromosomes
    grange.counts <- StringToGRanges(rownames(atac_counts), sep = c(":", "-"))
    grange.use <- seqnames(grange.counts) %in% standardChromosomes(grange.counts)
    atac_counts <- atac_counts[as.vector(grange.use), ]
    annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
    seqlevelsStyle(annotations) <- 'UCSC'
    genome(annotations) <- "hg38"

    chrom_assay <- CreateChromatinAssay(
       counts = atac_counts,
       sep = c(":", "-"),
       genome = 'hg38',
       fragments = sampleFile_atac,
       min.cells = 10,
       annotation = annotations
     )
    seurat[["ATAC"]] <- chrom_assay
    
    seurat[["percent.mt"]] <- PercentageFeatureSet(seurat, pattern = "^MT-")
    DefaultAssay(seurat) <- 'ATAC'
    seurat <- TSSEnrichment(seurat)
    p <- VlnPlot(seurat, features = c("nCount_ATAC", "nFeature_RNA", "nCount_RNA", "percent.mt", "TSS.enrichment"), ncol = 5)
    png(paste0(out_dir,'QC_VlnPlot_',sampleName,'.png'), res=300, height=7, width=14, units='in')
    print(p)
    dev.off()
    
    # Calculate quartile cutoffs
    quartiles <- qc_quartiles_multiome(seurat)
    lowerbound_nFeature <- quartiles[1]
    upperbound_nFeature <- quartiles[2]
    lowerbound_nCount <- quartiles[3]
    upperbound_nCount <- quartiles[4]
    lowerbound_percent.mt <- quartiles[5]
    upperbound_percent.mt <- quartiles[6]
    lowerbound_TSS <- quartiles[7]
    upperbound_TSS <- quartiles[8]
    
    # Add in qc data from per_barcode_metrics.csv
    qc <- read.table(file.path(inputFiles_dir[[i]], 'per_barcode_metrics.csv'), sep=',', header=TRUE, stringsAsFactors=1)
    rownames(qc) <- qc$gex_barcode
    qc <- qc[rownames(seurat@meta.data), ] # put them in the same row ordering as the cells ordered in seurat@meta.data
    seurat <- AddMetaData(seurat, qc)
    
    # QC plot - log10(Unique Fragments) vs TSS enrichment score
    # Pass-filter cells are identified as those cells having a TSS enrichment score greater than 4 and more than 1000 unique nuclear fragments. 
    log_data <- data.frame(genes=log10(seurat@meta.data$nFeature_RNA),fragments=log10(seurat@meta.data$atac_fragments),TSS=seurat@meta.data$TSS.enrichment)
    log_data$density <- get_density(log_data$fragments, log_data$TSS, n = 100)
    p <- ggplot(log_data) + geom_point(aes(fragments, TSS, color = density)) + scale_color_viridis(option = "H") +
             geom_hline(yintercept=2, linetype="dashed", color = "red") + # TSS Enrichment 2
             geom_vline(xintercept=log10(1000), linetype="dashed", color = "red") + # atac fragments threshold log10(1000)
             xlab('log10(fragments)') + ylab('TSS Enrichment') + ggtitle(paste0(sampleName)) +
             theme_minimal()
    png(paste0(out_dir,'QC_TSSxFragments_',sampleName,'.png'), res=300, height=7, width=7, units='in')
    print(p)
    dev.off()
    
    # QC plot - log10(RNA gene nFeatures) x log10(Fragments)
    p <- ggplot(log_data) + geom_point(aes(x=fragments, y=genes)) + 
             geom_hline(yintercept=log10(500), linetype="dashed", color = "red") + # RNA nFeatures threshold >= 500
             geom_vline(xintercept=log10(1000), linetype="dashed", color = "red") + # atac fragments threshold >= 1000
             xlab('log10(fragments)') + ylab('log10(RNA expressed genes)') + ggtitle(paste0(sampleName)) +
             theme_minimal()
    png(paste0(out_dir,'QC_GenexFragments_',sampleName,'.png'), res=300, height=7, width=7, units='in')
    print(p)
    dev.off()
    
    saveRDS(seurat, paste0(out_dir,sampleName,'_preQCfilt.rds'))
    
    # subset by QC filtering - can do this section automatically or manually adjusted QC parameters per sample. 
    # Just create and run separate code blocks for everything below per sample after QC filtering
    seurat_filtered <- subset(seurat, subset = 
                nFeature_RNA >= max(500,lowerbound_nFeature) & 
                nFeature_RNA <= upperbound_nFeature & 
                nCount_RNA <= upperbound_nCount & 
                percent.mt < 15 & # most lenient percent.mito we'll allow through. Can adjust down later
                atac_fragments >= 1000 & # 1000 is pretty default
                TSS.enrichment >= 2 & # archr uses tss.enrichment cutoff of 4. Signac calculates TSS differently and is usually lower so ~2.
                excluded_reason != 1 # this is cellranger's multiplet finder tag
                )
        
    # Filter atac counts matrix to the filtered cells and remake the chromatin assay
    atac_counts <- atac_counts[,colnames(seurat_filtered)]
    
    grange.counts <- StringToGRanges(rownames(atac_counts), sep = c(":", "-"))
    grange.use <- seqnames(grange.counts) %in% standardChromosomes(grange.counts)
    atac_counts <- atac_counts[as.vector(grange.use), ]
    annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
    seqlevelsStyle(annotations) <- 'UCSC'
    genome(annotations) <- "hg38"

    chrom_assay <- CreateChromatinAssay(
       counts = atac_counts,
       sep = c(":", "-"),
       genome = 'hg38',
       fragments = sampleFile_atac,
       min.cells = 10,
       annotation = annotations
     )
    seurat_filtered[["ATAC"]] <- chrom_assay
    
    print(paste0('After QC cutoffs: ', sampleName))
    print(seurat_filtered)

    filtered_list[[sampleName]] <- seurat_filtered
}

[1] "QC: D9_RV"


Genome matrix has multiple modalities, returning a list of matrices for this genome

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no 

[1] "lowerbound nFeature_RNA: -752.5"
[1] "upperbound nFeature_RNA: 4467.5"
[1] "lowerbound nCount_RNA: -3949.5"
[1] "upperbound nCount_RNA: 12198.5"
[1] "lowerbound percent.mt: 0"
[1] "upperbound percent.mt: 0"
[1] "lowerbound TSS.enrichment: 1.5"
[1] "upperbound TSS.enrichment: 5.5"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

[1] "After QC cutoffs: D9_RV"
An object of class Seurat 
156224 features across 7025 samples within 2 assays 
Active assay: ATAC (119623 features, 0 variable features)
 1 other assay present: RNA
[1] "QC: D2_RV"


...

[1] "After QC cutoffs: HF1_RV"
An object of class Seurat 
141661 features across 10342 samples within 2 assays 
Active assay: ATAC (105060 features, 0 variable features)
 1 other assay present: RNA
[1] "QC: HF3_RV"


Genome matrix has multiple modalities, returning a list of matrices for this genome

Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no 

[1] "lowerbound nFeature_RNA: -1245.5"
[1] "upperbound nFeature_RNA: 3390.5"
[1] "lowerbound nCount_RNA: -2936"
[1] "upperbound nCount_RNA: 6632"
[1] "lowerbound percent.mt: -1.5"
[1] "upperbound percent.mt: 2.5"
[1] "lowerbound TSS.enrichment: 1.5"
[1] "upperbound TSS.enrichment: 5.5"


Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warning message in .merge_two_Seqinfo_objects(x, y):
"The 2 combined objects have no sequence levels in common. (Use
  suppressWarnings() to suppress this warning.)"
Warn

[1] "After QC cutoffs: HF3_RV"
An object of class Seurat 
154720 features across 8613 samples within 2 assays 
Active assay: ATAC (118119 features, 0 variable features)
 1 other assay present: RNA


In [10]:
# seurat processing pipeline on each object after QC
for (i in seq_along(filtered_list)){
    sampleName <- names(filtered_list[i])
    seurat <- filtered_list[[i]]
    print(paste0('Processing: ', sampleName))
    
    # RNA analysis
    DefaultAssay(seurat) <- 'RNA'
    seurat <- NormalizeData(seurat, verbose = FALSE)
    seurat <- FindVariableFeatures(seurat, verbose = FALSE)
    seurat <- ScaleData(seurat, verbose = FALSE)
    seurat <- RunPCA(seurat, verbose = FALSE)
    seurat <- RunUMAP(seurat, dims = 1:50, reduction.name = 'umap.rna', reduction.key = 'rnaUMAP_')
    
    # ATAC analysis
    # We exclude the first dimension as this is typically correlated with sequencing depth
    DefaultAssay(seurat) <- 'ATAC'
    seurat <- RunTFIDF(seurat)
    seurat <- FindTopFeatures(seurat, min.cutoff='q0')
    seurat <- RunSVD(seurat)
    seurat <- RunUMAP(seurat, reduction='lsi', dims=2:50, reduction.name='umap.atac', reduction.key='atacUMAP_')
    
    # WNN multimodal analysis, representing a weighted combination of RNA and ATAC-seq modalities. 
    seurat <- FindMultiModalNeighbors(seurat, reduction.list=list('pca', 'lsi'), dims.list=list(1:50, 2:50))
    seurat <- RunUMAP(seurat, nn.name='weighted.nn', reduction.name='umap.wnn', reduction.key='wnnUMAP_')
    seurat <- FindClusters(seurat, graph.name='wsnn', verbose=FALSE)
    
    pdf(paste0(out_dir,'UMAPs_',sampleName,'_postQCfilt.pdf'), height=7, width=7)
    print(DimPlot(seurat, reduction = "umap.rna", label=TRUE, repel=TRUE))
    print(DimPlot(seurat, reduction = "umap.atac", label=TRUE, repel=TRUE))
    print(DimPlot(seurat, reduction = "umap.wnn", label=TRUE, repel=TRUE))
    dev.off()

    filtered_list[[sampleName]] <- seurat
    
    saveRDS(seurat, paste0(out_dir,sampleName,'_postQCfilt.rds'))
}

[1] "Processing: D9_RV"


Warning message:
"The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session"
22:07:12 UMAP embedding parameters a = 0.9922 b = 1.112

22:07:12 Read 7025 rows and found 50 numeric columns

22:07:12 Using Annoy for neighbor search, n_neighbors = 30

22:07:12 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

22:07:14 Writing NN index file to temp file /tmp/RtmpHtrdB2/file12f1c0cf1ec

22:07:14 Searching Annoy index using 1 thread, search_k = 3000

22:07:17 Annoy recall = 100%

22:07:20 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 

[1] "Processing: D2_RV"


22:11:58 UMAP embedding parameters a = 0.9922 b = 1.112

22:11:58 Read 7528 rows and found 50 numeric columns

22:11:58 Using Annoy for neighbor search, n_neighbors = 30

22:11:58 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

22:12:00 Writing NN index file to temp file /tmp/RtmpHtrdB2/file12f173f2680e

22:12:00 Searching Annoy index using 1 thread, search_k = 3000

22:12:02 Annoy recall = 100%

22:12:05 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

22:12:09 Initializing from normalized Laplacian + noise (using irlba)

22:12:10 Commencing optimization for 500 epochs, with 330292 positive edges

22:12:21 Optimization finished

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings

22:14:15 UMAP embedding parame

[1] "Processing: D3_RV"


...

22:33:06 UMAP embedding parameters a = 0.9922 b = 1.112

22:33:06 Read 12051 rows and found 50 numeric columns

22:33:06 Using Annoy for neighbor search, n_neighbors = 30

22:33:06 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

22:33:08 Writing NN index file to temp file /tmp/RtmpHtrdB2/file12f16be8d14

22:33:09 Searching Annoy index using 1 thread, search_k = 3000

22:33:13 Annoy recall = 100%

22:33:17 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

22:33:22 Initializing from normalized Laplacian + noise (using irlba)

22:33:23 Commencing optimization for 200 epochs, with 540226 positive edges

22:33:31 Optimization finished

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings

22:34:53 UMAP embedding parame

[1] "Processing: HF1_RV"


22:38:05 UMAP embedding parameters a = 0.9922 b = 1.112

22:38:05 Read 10342 rows and found 50 numeric columns

22:38:05 Using Annoy for neighbor search, n_neighbors = 30

22:38:05 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

22:38:07 Writing NN index file to temp file /tmp/RtmpHtrdB2/file12f156df10b1

22:38:07 Searching Annoy index using 1 thread, search_k = 3000

22:38:11 Annoy recall = 100%

22:38:14 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

22:38:19 Initializing from normalized Laplacian + noise (using irlba)

22:38:20 Commencing optimization for 200 epochs, with 459848 positive edges

22:38:28 Optimization finished

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings

22:39:46 UMAP embedding param

[1] "Processing: HF3_RV"


22:42:56 UMAP embedding parameters a = 0.9922 b = 1.112

22:42:56 Read 8613 rows and found 50 numeric columns

22:42:56 Using Annoy for neighbor search, n_neighbors = 30

22:42:56 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

22:42:58 Writing NN index file to temp file /tmp/RtmpHtrdB2/file12f121314d28

22:42:58 Searching Annoy index using 1 thread, search_k = 3000

22:43:01 Annoy recall = 100%

22:43:05 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

22:43:09 Initializing from normalized Laplacian + noise (using irlba)

22:43:10 Commencing optimization for 500 epochs, with 384610 positive edges

22:43:22 Optimization finished

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings

22:44:54 UMAP embedding parame

# Functions

In [8]:
# Calculate the lower and upper adjacent values of the nFeature VlnPlot for cutoffs
qc_quartiles_multiome <- function(seurat) {
    quartile1 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='nFeature_RNA'))[2])
    quartile1 <- as.integer(quartile1)

    quartile2 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='nFeature_RNA'))[5])
    quartile2 <- as.integer(quartile2)

    iqr <- quartile2 - quartile1

    lowerbound_nFeature <- quartile1 - (1.5 * iqr) # usually 1.5 * iqr. Use 2 to be more lax
    upperbound_nFeature <- quartile2 + (1.5 * iqr)

    print(paste0('lowerbound nFeature_RNA: ',lowerbound_nFeature))
    print(paste0('upperbound nFeature_RNA: ',upperbound_nFeature))
    
    quartile1 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='nCount_RNA'))[2])
    quartile1 <- as.integer(quartile1)

    quartile2 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='nCount_RNA'))[5])
    quartile2 <- as.integer(quartile2)

    iqr <- quartile2 - quartile1

    lowerbound_nCount <- quartile1 - (1.5 * iqr) # usually 1.5 * iqr. Use 2 to be more lax
    upperbound_nCount <- quartile2 + (1.5 * iqr)

    print(paste0('lowerbound nCount_RNA: ',lowerbound_nCount))
    print(paste0('upperbound nCount_RNA: ',upperbound_nCount))
    
    quartile1 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='percent.mt'))[2])
    quartile1 <- as.integer(quartile1)

    quartile2 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='percent.mt'))[5])
    quartile2 <- as.integer(quartile2)

    iqr <- quartile2 - quartile1

    lowerbound_percent.mt <- quartile1 - (1.5 * iqr) # usually 1.5 * iqr. Use 2 to be more lax
    upperbound_percent.mt <- quartile2 + (1.5 * iqr)

    print(paste0('lowerbound percent.mt: ',lowerbound_percent.mt))
    print(paste0('upperbound percent.mt: ',upperbound_percent.mt))
    
    # added TSS enrichment score section for multiome data
    quartile1 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='TSS.enrichment'))[2])
    quartile1 <- as.integer(quartile1)

    quartile2 <- gsub('.*:| ', '', summary(FetchData(seurat, vars='TSS.enrichment'))[5])
    quartile2 <- as.integer(quartile2)

    iqr <- quartile2 - quartile1

    lowerbound_TSS <- quartile1 - (1.5 * iqr) # usually 1.5 * iqr. Use 2 to be more lax
    upperbound_TSS <- quartile2 + (1.5 * iqr)

    print(paste0('lowerbound TSS.enrichment: ',lowerbound_TSS))
    print(paste0('upperbound TSS.enrichment: ',upperbound_TSS))
    
    l <- c(lowerbound_nFeature,upperbound_nFeature,lowerbound_nCount,upperbound_nCount,lowerbound_percent.mt,upperbound_percent.mt,lowerbound_TSS,upperbound_TSS)
    return(l)
}

In [7]:
get_density <- function(x, y, ...) {
  dens <- MASS::kde2d(x, y, ...)
  ix <- findInterval(x, dens$x)
  iy <- findInterval(y, dens$y)
  ii <- cbind(ix, iy)
  return(dens$z[ii])
}